In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import datetime

In [2]:
df = pd.read_csv("thursday.csv")
dfOldData = pd.read_csv("output.csv")

In [14]:
df.rt.unique()

array(['BB', 'NW', 'WX', 'MX', 'CN', 'CS', 'WS', 'NE', 'NX', 'DD', 'CSX'],
      dtype=object)

In [15]:
df.stpnm.unique()

array(['Baits II Inbound', 'Bursley Hall Inbound',
       'Hayward/Hubbard  Hubbard Outbound', 'Glen/Catherine Outbound',
       'Fuller Rd at Mitchell Field  Lot M-75',
       'Central Campus Transit Center: Ruthven Museum',
       'Pierpont Commons  Murfin Outbound', 'FXB Inbound',
       'Cooley Lab Inbound', 'Northwood I Outbound',
       'NCAC, Hubbard Outbound',
       'Fuller Rd at Lot NC-78  Mitchell Field (1)', 'Northwood V (3)',
       'Power Center', 'Glen/Catherine Inbound',
       'Pierpont Commons  Bonisteel Inbound', 'Cardiovascular Center',
       'NCAC  Hubbard Inbound', 'Hayward/Hubbard  Hubbard Inbound',
       'Plymouth Rd Crosswalk', 'Northwood Community Center',
       'Northwood III Outbound', 'Northwood II Outbound',
       'Stockwell Hall Outbound', 'Rackham Bldg',
       'Central Campus Transit Center: Chemistry', 'Couzens Hall',
       'Cancer Center Inbound', 'Mott Hospital', 'Markley Hall',
       'Taubman Outbound', 'Cancer Center Outbound', 'Taubman Inbou

In [3]:
def formatting(indata, stops, direction) :
    indata.tmstmp = pd.to_datetime(indata.tmstmp)
    stoptimes = indata[indata.rtdir.eq(direction) & indata.stpnm.isin(stops)]
    stoptimes = stoptimes.groupby([stoptimes.tmstmp.dt.date, "tatripid"]).agg("last")
    stoptimes.index.names = ["date", "tatripid"]
    return stoptimes

In [4]:
def WaittimesCalc(indata, stops, direction) :
    intimes = formatting(indata, stops, direction)
    waittimes = intimes.tmstmp.reset_index(level=1).groupby("date").agg(lambda x: x.sort_values().diff())
    waittimes = waittimes.reset_index()
    waittimes = pd.concat([waittimes.drop(['tmstmp'], axis=1), waittimes['tmstmp'].apply(pd.Series)], axis=1)
    waittimes = waittimes.melt(id_vars=["date"])
    waittimes = waittimes[["date", "value"]]
    waittimes["value"] = waittimes.value.dt.total_seconds()/60
    waittimes["zscores"] = (waittimes["value"] - waittimes["value"].mean())/waittimes["value"].std()
    waittimes = waittimes[waittimes.zscores < 3]
    return waittimes

In [5]:
def avgWaitCalc(indata, stops, direction) :
    waittimes = WaittimesCalc(indata, stops, direction)
    return waittimes.value.mean()

In [6]:
def avgWaitCalcRt(indata, stops, direction, routes) :
    rtData = indata[indata.rt.isin(routes)]
    return avgWaitCalc(rtData, stops, direction)

In [7]:
def graphWaittimes(indata, stops, direction) :
    waittimes = WaittimesCalc(indata, stops, direction)
    return sns.displot(waittimes.value)

In [8]:
avgWaitCalc(df, ["Central Campus Transit Center: Chemistry", "Central Campus Transit Center: Ruthven Museum"] , "NORTHBOUND")

2.6802911534154537

In [9]:
avgWaitCalcRt(df, ["Central Campus Transit Center: Chemistry", "Central Campus Transit Center: Ruthven Museum"] , "NORTHBOUND", ["BB"])

/Users/Angela/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


6.29842180774749

In [13]:
avgWaitCalcRt(df, ["Central Campus Transit Center: Chemistry", "Central Campus Transit Center: Ruthven Museum"] , "NORTHBOUND", ["DD"])

10.755905511811024

In [21]:
avgWaitCalcRt(df, ['Crisler Center/Lot SC-7'], 'NORTHBOUND', ['CN'])

/Users/Angela/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


10.587223587223587

In [12]:
# this isn't working
avgWaitCalc(dfOldData, ["Central Campus Transit Center: Chemistry"] , "NORTHBOUND")

AttributeError: Can only use .dt accessor with datetimelike values